In [5]:
import os
from PIL import Image
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score
import xml.etree.ElementTree as ET
import keras
from keras.applications.vgg16 import VGG16, preprocess_input
from keras.models import Model
from keras.layers import Dense, GlobalAveragePooling2D, Flatten
from tensorflow.keras.utils import img_to_array, load_img
from keras.utils import to_categorical
from timeit import default_timer as timer

In [3]:
# Get the current working directory
current_directory = os.getcwd()

# Access the parent folder
parent_folder = os.path.join(current_directory, 'streetsign')

#Create Images and Labels folders
image_folder = os.path.join(parent_folder, 'images')
labels_folder = os.path.join(parent_folder, 'annotations')

# Lists to store features (X) and labels (y)
X = []
y = []

for image_file in os.listdir(image_folder):
    # Load Image
    image_path = os.path.join(image_folder, image_file)
    image = load_img(image_path, target_size=(224, 224))
    image_array = img_to_array(image)

    X.append(image_array)

    # Extract corresponding label from the XML annotation
    label_file = os.path.join(labels_folder, image_file.replace('.png', '.xml'))
    tree = ET.parse(label_file)
    root = tree.getroot()
    class_label = root.find('.//object/name').text
    if class_label == 'speedlimit':
        class_label = 0
    elif class_label == 'stop':
        class_label = 1
    elif class_label == 'crosswalk':
        class_label = 2
    elif class_label == 'trafficlight':
        class_label = 3
    y.append(class_label)

# Converting lists to numpy arrays
X = np.array(X)
y = np.array(y)

pre_processed_X = []

for image in X:
  pre_processed_image = keras.applications.vgg16.preprocess_input(image)
  pre_processed_X.append(pre_processed_image)

# Spliting the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

num_classes = len(np.unique(y_train))
y_train_encoded = to_categorical(y_train, num_classes)
y_test_encoded = to_categorical(y_test, num_classes)

In [4]:
#load pre-trained model
model = VGG16(weights='imagenet', include_top=False, classes = 4, input_shape=(224, 224, 3))
x = model.output
x = GlobalAveragePooling2D()(x)  # Global average pooling layer
x = Dense(1024, activation='relu')(x)  # Fully connected layer
predictions = Dense(4, activation='softmax')(x)  # Output layer

model = Model(inputs=model.input, outputs=predictions)

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])


In [6]:
start = timer()
print("Starting Timer")
accuracy = model.evaluate(X_test, y_test_encoded)[1]
end = timer()
print("Time to Run: ", (end - start))
print(f"Accuracy: {accuracy}")

Starting Timer
6/6 [==============================] - 19s 3s/step - loss: 2.0297 - accuracy: 0.4261
Time to Run:  19.443746000000004
Accuracy: 0.4261363744735718


In [7]:
start = timer()
print("Starting Timer")
model.fit(X_train, y_train_encoded, epochs=10, batch_size=32)
end = timer()
print("Time to Run: ", (end - start))

Starting Timer
Epoch 1/10
22/22 [==============================] - 433s 20s/step - loss: 10.6713 - accuracy: 0.6462
Epoch 2/10
22/22 [==============================] - 468s 21s/step - loss: 0.8846 - accuracy: 0.7589
Epoch 3/10
22/22 [==============================] - 444s 20s/step - loss: 0.8795 - accuracy: 0.7589
Epoch 4/10
22/22 [==============================] - 436s 20s/step - loss: 0.8447 - accuracy: 0.7589
Epoch 5/10
22/22 [==============================] - 466s 21s/step - loss: 0.8262 - accuracy: 0.7589
Epoch 6/10
22/22 [==============================] - 405s 18s/step - loss: 0.8190 - accuracy: 0.7589
Epoch 7/10
22/22 [==============================] - 405s 19s/step - loss: 0.8135 - accuracy: 0.7589
Epoch 8/10
22/22 [==============================] - 421s 19s/step - loss: 0.8139 - accuracy: 0.7589
Epoch 9/10
22/22 [==============================] - 324s 15s/step - loss: 0.7807 - accuracy: 0.7589
Epoch 10/10
22/22 [==============================] - 320s 15s/step - loss: 0.7746 - 

In [8]:
start = timer()
print("Starting Timer")
accuracy = model.evaluate(X_test, y_test_encoded)[1]
end = timer()
print("Time to Run: ", (end - start))
print(f"Accuracy: {accuracy}")

Starting Timer
6/6 [==============================] - 18s 2s/step - loss: 0.9003 - accuracy: 0.6818
Time to Run:  23.48268829999961
Accuracy: 0.6818181872367859
